July 17th 2022
Version 1.0

This is an example of how to use the IBM QRadar API to close an offenses every 12 hours.
Educational purposes only. Please test it on your own risk. Do not test it in production environment.

Import the necessary modules

In [1]:
import requests
import json
import pandas as pd
import urllib.parse
import time
from pandas.io.json import json_normalize
# from pandas.json_normalize import json_normalize
# this line depends on the version of pandas you have installed
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

You can change the following variables to your own values

In [2]:
SEC_TOKEN = 'xxxxxxxxx-6ac6-4e93-xxxxxxx-19xxxxxxxb3cxxb'
URL_base = 'https://myqradar/api'
URL_suffix='/siem/offenses?filter=status%3Dopen'

In [3]:
URL_Concat = URL_base + URL_suffix

In order to get the data, we need to create a header with the security token

In [4]:
header = {
    'SEC':SEC_TOKEN,
    'Content-Type':'application/json',
    'accept':'application/json'
}

Define a funtion for get the list of open and active offenses

In [5]:
def get_offenses():
    URL = URL_base + URL_suffix
    response = requests.get(URL_Concat, headers=header, verify=False)
    if response.status_code == 200:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' success to get offenses\n')
        return response.json()
    else:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' Error: ' + str(response.status_code) + '\n')
        return None

Define a function to close an offense.
You need to change the reason id

In [6]:
def close_offense(offense_id):
    URL = URL_base + '/siem/offenses/' + str(offense_id) + '?closing_reason_id=54&status=CLOSED'
    response = requests.post(URL, headers=header, verify=False)
    if response.status_code == 200:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +  ' Offense ID: ' + str(offense_id) + ' closed' + '\n')
        return response.json()
    else:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' Error at close offense ' + offense_id + '. Error: ' + str(response.status_code) + '\n')
        return None

Define the function to add a comment to the offense. You need to change the note text

In [7]:
def add_comment(offense_id):
    URL = URL_base + '/siem/offenses/' + str(offense_id) + '/notes?note_text=Cerrado_despues_de_12_horas'
    response = requests.post(URL, headers=header, verify=False)
    if response.status_code == 201:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' Offense ID: ' + str(offense_id) + ' comment added' + '\n')
        return response.json()
    else:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' Error at add comment ' + offense_id + '. Error: ' + str(response.status_code) + '\n')
        return None

Use the function to get the list of active offenses and save it to a dataframe

In [8]:
offenses = get_offenses()
offenses=json_normalize(offenses)

This variable is used to get the actual time

In [9]:
today = pd.Timestamp.now()

For cicle to closea and comment offenses. If the offense was created more than 12 hours ago, it will be closed and a comment will be added

In [10]:
for i in range(len(offenses)):
    if offenses.iloc[i]['status'] == 'OPEN':
        time_offense = pd.to_datetime(offenses.iloc[i]['start_time'], unit='ms')
        diff = today - time_offense
        #timedelta is in days. .5 is 12 hours and 1 is 24 hours.Modify as needed.
        if diff > pd.Timedelta(days=1):
            add_comment(offenses.iloc[i]['id'])
            close_offense(offenses.iloc[i]['id'])
        else:
            with open ('close_offense.log', 'a') as f:
                f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' Offense ID: ' + str(offenses.iloc[i]['id']) + ' not closed, is not older than 24 hours' + '\n')
    else:
        with open ('close_offense.log', 'a') as f:
            f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +  ' Offense ID: ' + str(offenses.iloc[i]['id']) + ' is already closed' + '\n')

Offense 15679 is not older than 12 hours
Offense 15678 is not older than 12 hours
Offense 15677 is not older than 12 hours
Offense 15676 is not older than 12 hours
Offense 15675 is not older than 12 hours
Offense 15674 is not older than 12 hours
Offense 15673 is not older than 12 hours
Offense 15672 is not older than 12 hours
Offense 15671 is not older than 12 hours
Offense 15670 is not older than 12 hours
